In [ ]:
import logging

import config
import lightgbm as lgb
import polars as pl
import xgboost as xgb
from catboost import CatBoostClassifier, CatBoostRegressor
from lifelines import BreslowFlemingHarringtonFitter, KaplanMeierFitter

from src.customs.fold import add_kfold
from src.customs.metrics import CatBoostMetric, Metric, ROCAUCMetric, XGBMetric
from src.feature.tabular import AggregateEncoder, OrdinalEncoder, RawEncoder
from src.model.sklearn_like import (
    CatBoostClassifierWrapper,
    CatBoostRegressorWrapper,
    LightGBMWapper,
    XGBoostRegressorWrapper,
)
from src.trainer.tabular.simple import single_inference_fn, single_train_fn

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

In [ ]:
class CreateTargetFn:
    def __init__(self, time_col: str, event_col: str):
        self.time_col = time_col
        self.event_col = event_col
        self.target_cols = [
            "target_kmf_race_group",
            "target_kmf",
            "target_xgboost_cox",
            "target_bfhf",
        ]

    def __call__(self, df: pl.DataFrame) -> pl.DataFrame:
        # add kmf target by race_group
        dfs = []
        for _, i_df in df.group_by("race_group", maintain_order=True):
            kmf = KaplanMeierFitter()
            kmf.fit(i_df[self.time_col], event_observed=i_df[self.event_col])
            _df = i_df.with_columns(
                pl.Series("target_kmf_race_group", kmf.survival_function_at_times(i_df[self.time_col])),
            )
            dfs.append(_df)
        target_df = pl.concat(dfs)

        # add kmf target
        kmf = KaplanMeierFitter()
        kmf.fit(target_df[self.time_col], event_observed=target_df[self.event_col])
        target_df = target_df.with_columns(
            pl.Series("target_kmf", kmf.survival_function_at_times(target_df[self.time_col])),
        )

        # add xgboost cox target
        target_df = target_df.with_columns(
            pl.when(target_df[self.event_col] == 0)
            .then(-1 * target_df[self.time_col])
            .otherwise(target_df[self.time_col])
            .alias("target_xgboost_cox"),
        )
        # add bfhf target
        bfhf = BreslowFlemingHarringtonFitter()
        bfhf.fit(target_df[self.time_col], event_observed=target_df[self.event_col])
        target_df = target_df.with_columns(
            pl.Series("target_bfhf", bfhf.survival_function_at_times(target_df[self.time_col])),
        )

        return target_df


def load_data():
    raw_train_df = pl.read_csv(config.COMP_DATASET_DIR / "train.csv").with_columns(
        pl.lit("TRAIN").alias(config.DATASET_COL),
        pl.lit(-1).alias(config.FOLD_COL),
    )

    create_target_fn = CreateTargetFn(time_col=config.SURVIVAL_TIME_COL, event_col=config.EVENT_COL)
    raw_train_df = create_target_fn(raw_train_df)

    raw_test_df = pl.read_csv(config.COMP_DATASET_DIR / "test.csv").with_columns(
        pl.lit("TEST").alias(config.DATASET_COL)
    )

    config.META_COLS = sorted(list(set(config.META_COLS + create_target_fn.target_cols)))

    train_test_df = pl.concat([raw_train_df, raw_test_df], how="diagonal_relaxed").sort(
        config.DATASET_COL, config.ID_COL, descending=[True, False]
    )
    return train_test_df


def fe(
    train_test_df: pl.DataFrame,
    output_dataset: str = "TRAIN",
) -> pl.DataFrame:
    encoders = [
        RawEncoder(columns=config.META_COLS, prefix=""),
        RawEncoder(
            columns=(
                [
                    *config.NUMERICAL_COLS,
                ]
            ),
            prefix=f"{config.FEATURE_PREFIX}n_",
        ),
        OrdinalEncoder(
            columns=(
                [
                    *config.CATEGORICAL_COLS,
                ]
            ),
            prefix=f"{config.FEATURE_PREFIX}c_",
        ),
    ]

    for encoder in encoders:
        encoder.fit(train_test_df.filter(pl.col(config.DATASET_COL) == "TRAIN"))

    features_df = pl.concat(
        [
            encoder.transform(
                train_test_df.filter(pl.col(config.DATASET_COL) == output_dataset),
            )
            for encoder in encoders
        ],
        how="horizontal",
    )
    return features_df

In [ ]:
train_test_df = load_data()
features_df = fe(train_test_df)

feature_names = sorted([x for x in features_df.columns if x.startswith(config.FEATURE_PREFIX)])
cat_features = [x for x in feature_names if x.startswith(f"{config.FEATURE_PREFIX}c_")]


logger.info(f"# of features: {len(feature_names)}")
logger.info(f"# of cat_features: {len(cat_features)}")

In [ ]:
# base hazard
train_features_df = add_kfold(
    features_df,
    n_splits=5,
    random_state=config.N_SPLITS,
    fold_col=config.FOLD_COL,
)

va_result_df_cat, _, _ = single_train_fn(
    model=CatBoostRegressorWrapper(
        name="cat0",
        model=CatBoostRegressor(
            loss_function="Cox",
            learning_rate=0.08,
            n_estimators=20000,
            early_stopping_rounds=1000,
            eval_metric=CatBoostMetric(),
            verbose=100,
            # subsample=0.5,
            colsample_bylevel=0.2,
            random_state=config.SEED,
        ),
        multi_output=False,
        feature_names=feature_names,
        cat_features=cat_features,
    ),
    features_df=train_features_df,
    feature_cols=feature_names,
    target_col="target_xgboost_cox",
    fold_col=config.FOLD_COL,
    meta_cols=config.META_COLS,
    # weight_col="sample_weight",
    out_dir=config.TMP_DIR,
    train_folds=None,
    eval_fn=Metric(),
    overwrite=False,
    use_eval_metric_extra_va_df=True,
)

# va_result_df_xgb, _, _ = single_train_fn(
#     model=XGBoostRegressorWrapper(
#         name="xgb0",
#         model=xgb.XGBRegressor(
#             # booster="dart",
#             n_estimators=20000,
#             # max_depth=1,
#             subsample=0.5,
#             colsample_bytree=0.2,
#             # grow_policy="depthwise",
#             objective="survival:cox",
#             learning_rate=0.05,
#             eval_metric=XGBMetric(),
#         ),
#         # callbacks を引き継いでしまうので。。
#         early_stopping_params=dict(
#             rounds=500,
#             metric_name="XGBMetric",
#             data_name="validation_0",
#             save_best=True,
#             maximize=True,
#         ),
#         feature_names=feature_names,
#     ),
#     features_df=train_features_df,
#     feature_cols=feature_names,
#     target_col=config.EVENT_COL,
#     fold_col=config.FOLD_COL,
#     meta_cols=config.META_COLS,
#     out_dir=config.TMP_DIR,
#     train_folds=None,
#     eval_fn=Metric(),
#     overwrite=False,
#     use_eval_metric_extra_va_df=True,
# )

features_df = (
    features_df.join(
        va_result_df_cat.select(
            [
                config.ID_COL,
                pl.col("pred").alias("_pred_cat"),
                ((-(pl.col("target_kmf").log() * -1) + pl.col("pred")).exp()).alias("_target_cat"),
            ]
        ),
        on=config.ID_COL,
        how="inner",
    )
    # .join(
    #     va_result_df_xgb.select(
    #         [
    #             config.ID_COL,
    #             pl.col("pred").alias("_pred_xgb"),
    #             ((-(pl.col("target_kmf").log() * -1) + pl.col("pred")).exp()).alias("_target_xgb"),
    #         ]
    #     ),
    #     on=config.ID_COL,
    #     how="inner",
    # )
    .with_columns(
        pl.mean_horizontal(
            [
                "_target_cat",
                # "_target_xgb",  # hazard への変換方法を調査
            ]
        ).alias("target")
    )
)


In [ ]:
train_features_df = add_kfold(
    features_df,
    n_splits=config.N_SPLITS,
    random_state=config.N_SPLITS,
    fold_col=config.FOLD_COL,
)

va_result_df, va_scores, trained_models = single_train_fn(
    model=CatBoostRegressorWrapper(
        name="cat2",
        model=CatBoostRegressor(
            loss_function="Tweedie:variance_power=1.5",
            learning_rate=0.1,
            n_estimators=10000,
            early_stopping_rounds=1000,
            eval_metric=CatBoostMetric(),
            verbose=100,
            # subsample=0.5,
            colsample_bylevel=0.2,
            random_state=config.SEED,
        ),
        multi_output=False,
        feature_names=feature_names,
        cat_features=cat_features,
    ),
    features_df=train_features_df,
    feature_cols=feature_names,
    target_col="target",
    fold_col=config.FOLD_COL,
    meta_cols=config.META_COLS + ["target"],
    # weight_col="sample_weight",
    out_dir=config.OUTPUT_DIR,
    train_folds=None,
    eval_fn=Metric(),
    overwrite=True,
    use_eval_metric_extra_va_df=True,
)


In [ ]:
# debug test
test_features_df = fe(train_test_df, output_dataset="TEST")
te_result_df = single_inference_fn(
    model=CatBoostRegressorWrapper(name="cat2", feature_names=feature_names, cat_features=cat_features),
    features_df=test_features_df,
    feature_names=feature_names,
    model_dir=config.ARTIFACT_EXP_DIR(),
    inference_folds=list(range(config.N_SPLITS)),
    out_dir=config.OUTPUT_DIR,
)
print(te_result_df["pred"].to_list())